** Bayes Theorem**


```
 P(A/B) = P(B/A) * P(A)
          ---------------
               P(B)
```
Make a naive assumption that features are independent of each other.


### Import Libraries

In [ ]:
!pip install numpy pandas matplotlib seaborn --quiet

In [ ]:
!pip install opendatasets xgboost graphviz lightgbm scikit-learn xgboost lightgbm --upgrade --quiet

     |████████████████████████████████| 166.7 MB 22 kB/s 
     |████████████████████████████████| 2.0 MB 56.0 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 


In [ ]:
import os
import opendatasets as od
import pandas as pd
pd.set_option("display.max_columns", 120)
pd.set_option("display.max_rows", 120)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Load the data

In [ ]:
od.download('https://www.kaggle.com/c/titanic/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jahnavipravaleeka
Your Kaggle Key: ··········


100%|██████████| 34.1k/34.1k [00:00<00:00, 6.46MB/s]


Extracting archive ./titanic/titanic.zip to ./titanic


In [ ]:
os.listdir('titanic')

['gender_submission.csv', 'train.csv', 'test.csv']

In [ ]:
train = pd.read_csv('./titanic/train.csv')

In [ ]:
test = pd.read_csv('./titanic/test.csv')

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
len(train.columns)

12

### Data Exploration

In [ ]:
# Dropping unnecessary columns
train.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'], axis=1, inplace=True)
train.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [ ]:
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [ ]:
train['Sex'] = train['Sex'].replace({'male': 0, 'female': 1})

### Identify input and target columns

In [ ]:
X = train.drop(columns=['Survived'])
y = train['Survived']

In [ ]:
X.shape

(891, 4)

In [ ]:
y.shape

(891,)

###Separating numeric and categorical columns

In [ ]:
numeric_cols = X.select_dtypes(include = np.number).columns.tolist()
categorical_cols = X.select_dtypes(include = 'object').columns.tolist()

In [ ]:
print(numeric_cols)

['Pclass', 'Sex', 'Age', 'Fare']


In [ ]:
print(categorical_cols)

[]


### Impute missing data

In [ ]:
X.isna().sum()

Pclass    0
Sex       0
Age       0
Fare      0
dtype: int64

In [ ]:
X['Age'].describe()

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [ ]:
X['Age'] = X['Age'].fillna(X['Age'].mean())

### Scaling numeric columns to range (0,1)

In [ ]:
X[numeric_cols].describe().loc[['min','max']]

,Pclass,Sex,Age,Fare
min,1.0,0.0,0.42,0.0000
max,3.0,1.0,80.00,512.3292


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler().fit(train[numeric_cols])

In [ ]:
X[numeric_cols] = scaler.transform(X[numeric_cols])

In [ ]:
X[numeric_cols].describe().loc[['min','max']]

,Pclass,Sex,Age,Fare
min,0.0,0.0,0.0,0.0
max,1.0,1.0,1.0,1.0


### Splitting the Data into Train and Test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(712, 4)
(712,)
(179, 4)
(179,)


### Model Defining

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model = GaussianNB().fit(X_train, y_train)

In [ ]:
#train preds
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)

In [ ]:
model.score(X_val, y_val)

0.8044692737430168

### Preprocessing on test data

In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# Dropping unnecessary columns
test.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'], axis=1, inplace=True)
test.head()

,Pclass,Sex,Age,Fare
0,3,male,34.5,7.8292
1,3,female,47.0,7.0000
2,2,male,62.0,9.6875
3,3,male,27.0,8.6625
4,3,female,22.0,12.2875


In [ ]:
test['Sex'].value_counts()

male      266
female    152
Name: Sex, dtype: int64

In [ ]:
test['Sex'] = test['Sex'].replace({'male': 0, 'female': 1})

In [ ]:
test.isna().sum()

Pclass     0
Sex        0
Age       86
Fare       1
dtype: int64

In [ ]:
test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [ ]:
test[numeric_cols] = scaler.transform(test[numeric_cols])

In [ ]:
test_preds = model.predict(test)

In [ ]:
test_preds

array([0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [ ]:
submission = pd.read_csv('./titanic/gender_submission.csv')

In [ ]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [ ]:
submission['Survived'] = test_preds

In [ ]:
submission.to_csv('submission1.csv', index=False)

In [ ]:
from google.colab import files
download = files.download('submission1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>